In [ ]:
!pip install -q torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 torchtext==0.14.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu116 -U
!git lfs install
%cd /content
!git clone https://huggingface.co/camenduru/pocketsphinx-20.04-t4 pocketsphinx
%cd  /content/pocketsphinx
!sudo cmake --build build --target install
%cd /content
!git clone https://huggingface.co/camenduru/one-shot-talking-face-20.04-t4 one-shot-talking-face
%cd /content/one-shot-talking-face
!pip install -r /content/one-shot-talking-face/requirements.txt
!chmod 755 /content/one-shot-talking-face/OpenFace/FeatureExtraction
!mkdir /content/train
!apt install -qq libgtk2.0-0 jq -y
!pip install -q imageio-ffmpeg gradio numpy==1.23.0

Updated git hooks.
Git LFS initialized.
/content
fatal: destination path 'pocketsphinx' already exists and is not an empty directory.
/content/pocketsphinx
Consolidate compiler generated dependencies of target pocketsphinx
[ 88%] Built target pocketsphinx
Consolidate compiler generated dependencies of target pocketsphinx_main
[ 88%] Built target pocketsphinx_main
Consolidate compiler generated dependencies of target pocketsphinx_batch
[ 88%] Built target pocketsphinx_batch
Consolidate compiler generated dependencies of target pocketsphinx_mdef_convert
[ 90%] Built target pocketsphinx_mdef_convert
Consolidate compiler generated dependencies of target pocketsphinx_jsgf2fsg
[ 93%] Built target pocketsphinx_jsgf2fsg
Consolidate compiler generated dependencies of target pocketsphinx_lm_convert
[ 95%] Built target pocketsphinx_lm_convert
Consolidate compiler generated dependencies of target pocketsphinx_lm_eval
[ 97%] Built target pocketsphinx_lm_eval
Consolidate compiler generated dependenc

In [ ]:
!wget https://github.com/camenduru/one-shot-talking-face-colab/raw/main/test/audio.wav -O /content/audio.wav
!wget https://github.com/camenduru/one-shot-talking-face-colab/blob/main/test/image.png?raw=true -O /content/image.png

--2024-12-28 10:17:54--  https://github.com/camenduru/one-shot-talking-face-colab/raw/main/test/audio.wav
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/camenduru/one-shot-talking-face-colab/main/test/audio.wav [following]
--2024-12-28 10:17:54--  https://raw.githubusercontent.com/camenduru/one-shot-talking-face-colab/main/test/audio.wav
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 608334 (594K) [audio/wav]
Saving to: ‘/content/audio.wav’

/content/audio.wav  100%[===================>] 594.08K  --.-KB/s    in 0.02s   

2024-12-28 10:17:54 (32.2 MB/s) - ‘/content/audio.wav’ saved [608334/6083

In [ ]:
import os
import torchaudio
import gradio as gr
from IPython.display import HTML
from base64 import b64encode

# Function to display the generated video in HTML format
def show_video(video_path, video_width=256):
    try:
        print(f"Attempting to load video from: {video_path}")
        video_file = open(video_path, "rb").read()
        video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
        return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")
    except Exception as e:
        print(f"Error loading video: {str(e)}")
        return f"Error loading video: {str(e)}"

# Main processing function
def process_files(audio_file_path, image_file_path):
    wav_file = "/content/audio.wav"
    image_path = "/content/image1.png"

    try:
        # Copy audio file to target location
        print("Copying uploaded audio file...")
        os.system(f"cp {audio_file_path} {wav_file}")

        # Copy image file to target location
        print("Copying uploaded image file...")
        os.system(f"cp {image_file_path} {image_path}")

        # Load audio file to validate and re-save it
        print("Validating and processing audio file...")
        waveform, sample_rate = torchaudio.load(wav_file)
        torchaudio.save(wav_file, waveform, sample_rate, encoding="PCM_S", bits_per_sample=16)

        # Create required directories
        print("Creating required directories...")
        #os.makedirs("/content/train", exist_ok=True)
        #os.system(f"cp {wav_file} /content/train/audio.wav")
        #os.system(f"cp {image_path} /content/train/image.png")

        # Run PocketSphinx for phoneme alignment
        print("Running PocketSphinx for phoneme alignment...")
        command = (
            f"pocketsphinx -phone_align yes single /content/audio.wav $text | "
            f"jq '[.w[]|{{word: (.t | ascii_upcase | sub(\"<S>\"; \"sil\") | "
            f"sub(\"<SIL>\"; \"sil\") | sub(\"\\(2\\)\"; \"\") | "
            f"sub(\"\\(3\\)\"; \"\") | sub(\"\\(4\\)\"; \"\") | "
            f"sub(\"\\[SPEECH\\]\"; \"SIL\") | sub(\"\\[NOISE\\]\"; \"SIL\")), "
            f"phones: [.w[]|{{ph: .t | sub(\"\\+SPN\\+\"; \"SIL\") | "
            f"sub(\"\\+NSN\\+\"; \"SIL\"), bg: (.b*100)|floor, ed: (.b*100+.d*100)|floor}}]}}]' > /content/test.json"
        )
        print(f"Executing command: {command}")
        os.system(command)

        # Run the one-shot talking face script
        print("Running the one-shot talking face script...")
        os.chdir("/content/one-shot-talking-face")
        os.system(f"python -B test_script.py --img_path /content/image.png --audio_path /content/audio.wav --phoneme_path /content/test.json --save_dir /content/")

        # Return the generated video path
        generated_video_path = "/content/image_audio.mp4"
        print(f"Generated video saved at: {generated_video_path}")
        return generated_video_path

    except Exception as e:
        print(f"Error processing files: {str(e)}")
        return f"Error processing files: {str(e)}"

# Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("## One-Shot Talking Face Generator")

    with gr.Row():
        audio_input = gr.File(label="Upload Audio File", type="filepath")
        image_input = gr.File(label="Upload Image File", type="filepath")

    generate_button = gr.Button("Generate Video")
    output_video = gr.Video(label="Generated Video")

    # Function to handle the button click
    def generate_and_display(audio_file, image_file):
        try:
            print(f"Received files - Audio: {audio_file.name}, Image: {image_file.name}")
            video_path = process_files(audio_file.name, image_file.name)
            if isinstance(video_path, str) and video_path.startswith("/content/"):
                print(f"Successfully generated video: {video_path}")
                return video_path  # Return the video path if successful
            else:
                print(f"Error or invalid video path returned: {video_path}")
                return video_path  # Return the error message if not

        except Exception as e:
            print(f"Error generating video: {str(e)}")
            return f"Error generating video: {str(e)}"

    # Set up the button click callback
    generate_button.click(generate_and_display, inputs=[audio_input, image_input], outputs=output_video)

# Launch the Gradio app
print("Launching Gradio app...")
demo.launch(debug=True,allowed_paths=["/content/"])


Launching Gradio app...
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://6b422030f9b7c6d0bc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
